In [2]:
# Setup for performance metrics - create nested directories
import random
from time import sleep
perf_base = "spytial_perf"
def get_perf_path(structure, size):
    return perf_base + "_" + structure + "_" + f"{size}.json"
PI = 20
SIZES = [5, 10, 25, 50]


In [3]:
from spytial import *
from spytial.annotations import *

# Binary Search Trees

In [18]:
# Convention: Nodes keys are visualized within the node.
@attribute(field="key")
# Conception:  Left child is to the left and BELOW a node.
@orientation(selector='left & (BSTNode -> (BSTNode - NoneType.~key))',
             directions=['below','left'])
# Conception: Right child is to the right and BELOW a node.
@orientation(selector='right & (BSTNode -> (BSTNode - NoneType.~key))',
             directions=['below','right'])
class BSTNode:
    def __init__(self, key=None, left=None, right=None, parent=None):
        self.key = key
        self.left = left
        self.right = right
        self.parent = parent

# Singleton NIL sentinel
BST_NIL = BSTNode(key=None)
BST_NIL.left = BST_NIL.right = BST_NIL.parent = BST_NIL


# Convention: Hiding extra details for clarity
@hideAtom(selector='{ x : BSTNode | (x.key in NoneType) } + BSTree + int + NoneType')   
@hideField(field='parent')  
class BSTree:
    def __init__(self):
        self.root = BST_NIL

    # TREE-SEARCH
    def search(self, k):
        x = self.root
        while x is not BST_NIL and k != x.key:
            x = x.left if k < x.key else x.right
        return x

    # TREE-MINIMUM / MAXIMUM
    def minimum(self, x):
        while x.left is not BST_NIL:
            x = x.left
        return x
    def maximum(self, x):
        while x.right is not BST_NIL:
            x = x.right
        return x

    # TREE-SUCCESSOR
    def successor(self, x):
        if x.right is not BST_NIL:
            return self.minimum(x.right)
        y = x.parent
        while y is not BST_NIL and x is y.right:
            x, y = y, y.parent
        return y

    # TREE-INSERT
    def insert(self, k):
        z = BSTNode(key=k, left=BST_NIL, right=BST_NIL, parent=None)
        y, x = BST_NIL, self.root
        while x is not BST_NIL:
            y = x
            x = x.left if z.key < x.key else x.right
        z.parent = y
        if y is BST_NIL: self.root = z
        elif z.key < y.key: y.left = z
        else: y.right = z
        return z

    # helper: TRANSPLANT
    def _transplant(self, u, v):
        if u.parent is BST_NIL: self.root = v
        elif u is u.parent.left: u.parent.left = v
        else: u.parent.right = v
        v.parent = u.parent

    # TREE-DELETE
    def delete(self, z):
        if z.left is BST_NIL:
            self._transplant(z, z.right)
        elif z.right is BST_NIL:
            self._transplant(z, z.left)
        else:
            y = self.minimum(z.right)     # successor
            if y.parent is not z:
                self._transplant(y, y.right)
                y.right = z.right
                y.right.parent = y
            self._transplant(z, y)
            y.left = z.left
            y.left.parent = y

    # inorder (for sanity checks)
    def inorder(self, node=None, acc=None):
        if node is None:
            node, acc = self.root, []
        if node is BST_NIL:
            return acc
        self.inorder(node.left, acc)
        acc.append(node.key)
        self.inorder(node.right, acc)
        return acc


![bst](./img/binary-search-tree.png)

In [19]:

t = BSTree()

# insert keys (any iterable of comparable keys works)
for k in [15, 6, 18, 17, 20, 3, 7, 13, 9, 2, 4]:
    t.insert(k)

diagram(t) 

## Performance Data

This subsection is for perf data. **This will oepn your browser and save values to your computer

In [20]:
STRUCTURE = "bstree"
for size in SIZES:
    values = random.sample(range(1, 1000), size)

    bst = BSTree()
    for val in values:
        bst.insert(val)

    diagram(bst, method="browser", perf_path=get_perf_path(STRUCTURE, size), perf_iterations=PI, headless=True)

Running 20 iterations (timeout: 120s)...
  Progress: 20/20 iterations (10.0s elapsed)
✓ Headless benchmark completed: 20 iterations
  Generate Layout: 9.99ms avg
  Render Layout: 28.21ms avg
  Total Time: 38.82ms avg
  Metrics saved to: spytial_perf_bstree_5.json
Running 20 iterations (timeout: 120s)...
  Progress: 20/20 iterations (10.0s elapsed)
✓ Headless benchmark completed: 20 iterations
  Generate Layout: 11.44ms avg
  Render Layout: 45.90ms avg
  Total Time: 58.29ms avg
  Metrics saved to: spytial_perf_bstree_10.json
Running 20 iterations (timeout: 120s)...
  Progress: 20/20 iterations (10.0s elapsed)
✓ Headless benchmark completed: 20 iterations
  Generate Layout: 11.88ms avg
  Render Layout: 100.38ms avg
  Total Time: 113.55ms avg
  Metrics saved to: spytial_perf_bstree_25.json
Running 20 iterations (timeout: 120s)...
  Progress: 20/20 iterations (10.0s elapsed)
✓ Headless benchmark completed: 20 iterations
  Generate Layout: 15.60ms avg
  Render Layout: 247.13ms avg
  Total T

# Red Black Trees

In [4]:
RED = "red"
BLACK = "black"



@orientation(selector='left & (RBNode -> (RBNode - NoneType.~key))', directions=['below', 'left'])
@orientation(selector='right & (RBNode -> (RBNode - NoneType.~key))', directions=['below', 'right'])
# Conception: Red / Black nodes are distinguished, (red and black are ~conception ~ convention)
@atomColor(
    selector='{ x : RBNode | @:(x.color) = red }',
    value='red'
)
@atomColor(
    selector='{ x : RBNode | @:(x.color) = black }',
    value='black'
)
# Convention: Nodes keys are visualized within the node.
@attribute(field="key")
@attribute(field="color")
@hideAtom(selector='{ x : RBNode | (x.key in NoneType) } + RBTree + int + NoneType + {s : str | @:s = red or @:s = black}') # Hide the NIL node
@hideField(field='parent') 
class RBNode:
    def __init__(self, key=None, color=BLACK, left=None, right=None, parent=None):
        self.key = key
        self.color = color
        self.left = left
        self.right = right
        self.parent = parent

# Singleton NIL sentinel (all leaves point here)
NIL = RBNode(key=None, color=BLACK)
NIL.left = NIL.right = NIL.parent = NIL


class RBTree:
    def __init__(self):
        self.root = NIL

    # ----- Utility -----
    def search(self, key):
        x = self.root
        while x is not NIL and key != x.key:
            x = x.left if key < x.key else x.right
        return x

    def minimum(self, x):
        while x.left is not NIL:
            x = x.left
        return x

    # ----- Rotations (CLRS §13.2) -----
    def left_rotate(self, x):
        y = x.right
        assert y is not NIL, "left_rotate requires x.right != NIL"
        x.right = y.left
        if y.left is not NIL:
            y.left.parent = x
        y.parent = x.parent
        if x.parent is NIL:
            self.root = y
        elif x is x.parent.left:
            x.parent.left = y
        else:
            x.parent.right = y
        y.left = x
        x.parent = y

    def right_rotate(self, y):
        x = y.left
        assert x is not NIL, "right_rotate requires y.left != NIL"
        y.left = x.right
        if x.right is not NIL:
            x.right.parent = y
        x.parent = y.parent
        if y.parent is NIL:
            self.root = x
        elif y is y.parent.left:
            y.parent.left = x
        else:
            y.parent.right = x
        x.right = y
        y.parent = x

    # ----- Insert + Fixup (CLRS §13.3) -----
    def insert(self, key):
        z = RBNode(key=key, color=RED, left=NIL, right=NIL, parent=None)
        y = NIL
        x = self.root
        # BST insert
        while x is not NIL:
            y = x
            x = x.left if z.key < x.key else x.right
        z.parent = y
        if y is NIL:
            self.root = z
        elif z.key < y.key:
            y.left = z
        else:
            y.right = z
        # Fix red-black properties
        self._insert_fixup(z)
        return z

    def _insert_fixup(self, z):
        while z.parent.color is RED:
            if z.parent is z.parent.parent.left:
                y = z.parent.parent.right  # uncle
                if y.color is RED:
                    # Case 1
                    z.parent.color = BLACK
                    y.color = BLACK
                    z.parent.parent.color = RED
                    z = z.parent.parent
                else:
                    if z is z.parent.right:
                        # Case 2
                        z = z.parent
                        self.left_rotate(z)
                    # Case 3
                    z.parent.color = BLACK
                    z.parent.parent.color = RED
                    self.right_rotate(z.parent.parent)
            else:
                # mirror image
                y = z.parent.parent.left
                if y.color is RED:
                    z.parent.color = BLACK
                    y.color = BLACK
                    z.parent.parent.color = RED
                    z = z.parent.parent
                else:
                    if z is z.parent.left:
                        z = z.parent
                        self.right_rotate(z)
                    z.parent.color = BLACK
                    z.parent.parent.color = RED
                    self.left_rotate(z.parent.parent)
        self.root.color = BLACK

    # ----- (Optional) Traversal for testing -----
    def inorder(self, node=None, acc=None):
        if node is None:
            node, acc = self.root, []
        if node is NIL:
            return acc
        self.inorder(node.left, acc)
        acc.append((node.key, node.name))
        self.inorder(node.right, acc)
        return acc


![rbt-3-ways](./img/red-black-tree.png)

In [ ]:
t = RBTree()
for k in [26, 17, 41, 47, 30, 38, 35, 39, 28, 21, 23, 19, 20, 14, 16, 15, 10, 7, 3, 12]:
    t.insert(k)
diagram(t) # Shows you each step of the insertion

## Performance

In [23]:
STRUCTURE = "rbtree"
for size in SIZES:
    values = random.sample(range(1, 1000), size)

    bst = RBTree()
    for val in values:
        bst.insert(val)

    diagram(bst, method="browser", perf_path=get_perf_path(STRUCTURE, size), perf_iterations=PI, headless=True)


Running 20 iterations (timeout: 120s)...
  Progress: 20/20 iterations (10.0s elapsed)
✓ Headless benchmark completed: 20 iterations
  Generate Layout: 17.08ms avg
  Render Layout: 29.87ms avg
  Total Time: 48.06ms avg
  Metrics saved to: spytial_perf_rbtree_5.json
Running 20 iterations (timeout: 120s)...
  Progress: 20/20 iterations (10.0s elapsed)
✓ Headless benchmark completed: 20 iterations
  Generate Layout: 16.95ms avg
  Render Layout: 45.91ms avg
  Total Time: 64.20ms avg
  Metrics saved to: spytial_perf_rbtree_10.json
Running 20 iterations (timeout: 120s)...
  Progress: 20/20 iterations (10.0s elapsed)
✓ Headless benchmark completed: 20 iterations
  Generate Layout: 17.38ms avg
  Render Layout: 104.35ms avg
  Total Time: 123.65ms avg
  Metrics saved to: spytial_perf_rbtree_25.json
Running 20 iterations (timeout: 120s)...
  Progress: 20/20 iterations (10.0s elapsed)
✓ Headless benchmark completed: 20 iterations
  Generate Layout: 21.96ms avg
  Render Layout: 250.40ms avg
  Total 

# Order Statistic Trees (Augmented RB Trees)

In [24]:

@attribute(field="size")
class OSNode(RBNode):
    def __init__(self, key, color=RED, left=None, right=None, parent=None):
        super().__init__(key=key, color=color,
                         left=left if left is not None else NIL,
                         right=right if right is not None else NIL,
                         parent=parent if parent is not None else NIL)
        self.size = 1

class OSTree(RBTree):
    def __init__(self):
        super().__init__()
        if not hasattr(NIL, "size"):
            setattr(NIL, "size", 0)

    # ----- augmentation helpers -----
    def _update(self, x: OSNode):
        if x is NIL: return
        x.size = 1 + (x.left.size if x.left is not NIL else 0) + (x.right.size if x.right is not NIL else 0)

    def _update_up(self, x: OSNode):
        while x is not NIL:
            old = x.size
            self._update(x)
            if x.size == old:
                break
            x = x.parent

    # ----- rotations with size maintenance -----
    def left_rotate(self, x: OSNode):
        y = x.right
        assert y is not NIL
        x.right = y.left
        if y.left is not NIL:
            y.left.parent = x
        y.parent = x.parent
        if x.parent is NIL:
            self.root = y
        elif x is x.parent.left:
            x.parent.left = y
        else:
            x.parent.right = y
        y.left = x
        x.parent = y
        # update sizes bottom-up
        self._update(x)
        self._update(y)
        self._update_up(y.parent)

    def right_rotate(self, y: OSNode):
        x = y.left
        assert x is not NIL
        y.left = x.right
        if x.right is not NIL:
            x.right.parent = y
        x.parent = y.parent
        if y.parent is NIL:
            self.root = x
        elif y is y.parent.left:
            y.parent.left = x
        else:
            y.parent.right = x
        x.right = y
        y.parent = x
        self._update(y)
        self._update(x)
        self._update_up(x.parent)

    # ----- insert (creates OSNode, maintains size) -----
    def insert(self, key):
            z = OSNode(key=key, color=RED)
            y = NIL
            x = self.root
            while x is not NIL:
                y = x
                x = x.left if z.key < x.key else x.right
            z.parent = y
            if y is NIL:
                self.root = z
            elif z.key < y.key:
                y.left = z
            else:
                y.right = z

            # update sizes on the insertion path
            self._update_up(z)

            # fix red-black properties (may rotate / change structure)
            self._insert_fixup(z)

            # ensure sizes are correct after possible rotations/re-parenting:
            # z may have moved during fixup, so update up from z and from its parent.
            self._update_up(z)
            if z.parent is not NIL:
                self._update_up(z.parent)

            return z
    # ----- delete (expects a node handle) -----
    def delete(self, z: OSNode):
        def transplant(u, v):
            if u.parent is NIL:
                self.root = v
            elif u is u.parent.left:
                u.parent.left = v
            else:
                u.parent.right = v
            v.parent = u.parent

        def minimum(x):
            while x.left is not NIL:
                x = x.left
            return x

        y = z
        y_orig_color = y.color
        if z.left is NIL:
            x = z.right
            transplant(z, z.right)
            start = x.parent
        elif z.right is NIL:
            x = z.left
            transplant(z, z.left)
            start = x.parent
        else:
            y = minimum(z.right)
            y_orig_color = y.color
            x = y.right
            if y.parent is z:
                x.parent = y
            else:
                transplant(y, y.right)
                y.right = z.right
                y.right.parent = y
            transplant(z, y)
            y.left = z.left
            y.left.parent = y
            y.color = z.color
            self._update(y)
            start = y

        self._update_up(start)  # fix sizes after structure change

        if y_orig_color is BLACK:
            self._delete_fixup(x)

    def _delete_fixup(self, x: OSNode):
        while x is not self.root and x.color is BLACK:
            if x is x.parent.left:
                w = x.parent.right
                if w.color is RED:
                    w.color = BLACK
                    x.parent.color = RED
                    self.left_rotate(x.parent)
                    w = x.parent.right
                if w.left.color is BLACK and w.right.color is BLACK:
                    w.color = RED
                    x = x.parent
                else:
                    if w.right.color is BLACK:
                        w.left.color = BLACK
                        w.color = RED
                        self.right_rotate(w)
                        w = x.parent.right
                    w.color = x.parent.color
                    x.parent.color = BLACK
                    w.right.color = BLACK
                    self.left_rotate(x.parent)
                    x = self.root
            else:
                w = x.parent.left
                if w.color is RED:
                    w.color = BLACK
                    x.parent.color = RED
                    self.right_rotate(x.parent)
                    w = x.parent.left
                if w.right.color is BLACK and w.left.color is BLACK:
                    w.color = RED
                    x = x.parent
                else:
                    if w.left.color is BLACK:
                        w.right.color = BLACK
                        w.color = RED
                        self.left_rotate(w)
                        w = x.parent.left
                    w.color = x.parent.color
                    x.parent.color = BLACK
                    w.left.color = BLACK
                    self.right_rotate(x.parent)
                    x = self.root
        x.color = BLACK
        self._update_up(x)  # restore sizes

    # ----- order-statistic ops -----
    # 1-indexed select: return node with rank i
    def os_select(self, i: int) -> OSNode | None:
        x = self.root
        while x is not NIL:
            r = (x.left.size if x.left is not NIL else 0) + 1
            if i == r:
                return x
            elif i < r:
                x = x.left
            else:
                i -= r
                x = x.right
        return None

    # rank of node x (1-indexed)
    def os_rank(self, x: OSNode) -> int:
        r = (x.left.size if x.left is not NIL else 0) + 1
        y = x
        while y is not self.root:
            if y is y.parent.right:
                r += (y.parent.left.size if y.parent.left is not NIL else 0) + 1
            y = y.parent
        return r


![order-statistic](./img/augmentation-order-statistic-tree.png)

In [25]:
t = OSTree()
nodes = {}
for k in [26, 17, 41, 14, 21, 16, 19]:
    nodes[k] = t.insert(k)
diagram(t)

## Performance

In [26]:
STRUCTURE = "ostree"
for size in SIZES:
    values = random.sample(range(1, 1000), size)

    bst = OSTree()
    for val in values:
        bst.insert(val)

    diagram(bst, method="browser", perf_path=get_perf_path(STRUCTURE, size), perf_iterations=PI, headless=True)

Running 20 iterations (timeout: 120s)...
  Progress: 20/20 iterations (10.0s elapsed)
✓ Headless benchmark completed: 20 iterations
  Generate Layout: 14.02ms avg
  Render Layout: 26.52ms avg
  Total Time: 41.36ms avg
  Metrics saved to: spytial_perf_ostree_5.json
Running 20 iterations (timeout: 120s)...
  Progress: 20/20 iterations (10.0s elapsed)
✓ Headless benchmark completed: 20 iterations
  Generate Layout: 17.15ms avg
  Render Layout: 45.38ms avg
  Total Time: 63.97ms avg
  Metrics saved to: spytial_perf_ostree_10.json
Running 20 iterations (timeout: 120s)...
  Progress: 20/20 iterations (10.0s elapsed)
✓ Headless benchmark completed: 20 iterations
  Generate Layout: 18.12ms avg
  Render Layout: 103.94ms avg
  Total Time: 124.20ms avg
  Metrics saved to: spytial_perf_ostree_25.json
Running 20 iterations (timeout: 120s)...
  Progress: 20/20 iterations (10.0s elapsed)
✓ Headless benchmark completed: 20 iterations
  Generate Layout: 23.38ms avg
  Render Layout: 250.19ms avg
  Total 

# B Trees

In [27]:
from __future__ import annotations
from dataclasses import dataclass, field
from typing import List, Optional, Tuple, Any

## Selector that gives us parent -> idx -> child 
pic = "children.idx" #"{p: BTreeNode, i : int, c : BTreeNode| c in (((p.children).idx)[i]) }"

## WE WANT PARENT->CHILD AS WELL.
pc =  "{p: BTreeNode, c : BTreeNode| c in (((p.children).idx)[int]) }"

ALIGNED = f"((~{pc}).{pc})" # f"{{ c1, c2 :  BTreeNode | (some p : BTreeNode | ((p->c1) + (p->c2))  in {pc}   )}}"

# ------------------------- Node -------------------------
@dataclass
@attribute(field="t")
@attribute(field="leaf")
@hideAtom(selector="list + bool + int")
@group(selector="{p : BTreeNode, k : str | k in (((p.keys).idx)[int]) } + {p, pp : BTreeNode | p = pp}", name="keys") ## ((p.keys).idx[int]) + (iden & (BTreeNode -> BTreeNode))
@align(selector="{p : BTreeNode, k : str | k in (((p.keys).idx)[int]) } + {p, pp : BTreeNode | p = pp}", direction="horizontal") ## (p.keys).idx[int] + (iden & (BTreeNode -> BTreeNode))
@inferredEdge(selector=pic, name="child")
@orientation(selector=pic, directions=["below"])
@align(selector=ALIGNED, direction="horizontal") ## This?
class BTreeNode:
    t: int
    leaf: bool = True
    keys: List[Any] = field(default_factory=list)           # sorted
    children: List["BTreeNode"] = field(default_factory=list)  # len = len(keys)+1 if !leaf

    # SEARCH (B-TREE-SEARCH)
    def search(self, k: Any) -> Optional[Tuple["BTreeNode", int]]:
        i = 0
        while i < len(self.keys) and k > self.keys[i]:
            i += 1
        if i < len(self.keys) and k == self.keys[i]:
            return (self, i)
        if self.leaf:
            return None
        return self.children[i].search(k)

    # SPLIT-CHILD (B-TREE-SPLIT-CHILD)
    def split_child(self, i: int) -> None:
        t = self.t
        y = self.children[i]
        z = BTreeNode(t=t, leaf=y.leaf)

        mid = y.keys[t-1]
        z.keys = y.keys[t:]            # upper t-1 keys
        y.keys = y.keys[:t-1]          # lower t-1 keys

        if not y.leaf:
            z.children = y.children[t:]
            y.children = y.children[:t]

        self.children.insert(i+1, z)
        self.keys.insert(i, mid)

    # INSERT-NONFULL (B-TREE-INSERT-NONFULL)
    def insert_nonfull(self, k: Any) -> None:
        i = len(self.keys) - 1
        if self.leaf:
            self.keys.append(k)
            j = len(self.keys) - 1
            while j > 0 and self.keys[j-1] > k:
                self.keys[j] = self.keys[j-1]
                j -= 1
            self.keys[j] = k
            return

        while i >= 0 and k < self.keys[i]:
            i -= 1
        i += 1
        if len(self.children[i].keys) == 2*self.t - 1:
            self.split_child(i)
            if k > self.keys[i]:
                i += 1
        self.children[i].insert_nonfull(k)

    # ----- DELETE helpers -----
    def _pred(self, idx: int) -> Any:
        cur = self.children[idx]
        while not cur.leaf:
            cur = cur.children[-1]
        return cur.keys[-1]

    def _succ(self, idx: int) -> Any:
        cur = self.children[idx+1]
        while not cur.leaf:
            cur = cur.children[0]
        return cur.keys[0]

    def _merge(self, idx: int) -> None:
        """Merge children[idx], keys[idx], children[idx+1] into children[idx]."""
        child = self.children[idx]
        sibling = self.children[idx+1]
        child.keys.append(self.keys[idx])
        child.keys.extend(sibling.keys)
        if not child.leaf:
            child.children.extend(sibling.children)
        del self.keys[idx]
        del self.children[idx+1]

    def _fill(self, idx: int) -> None:
        """Ensure children[idx] has at least t keys."""
        t = self.t
        if idx > 0 and len(self.children[idx-1].keys) >= t:
            # Borrow from left
            left = self.children[idx-1]
            child = self.children[idx]
            child.keys.insert(0, self.keys[idx-1])
            if not child.leaf:
                child.children.insert(0, left.children.pop())
            self.keys[idx-1] = left.keys.pop()
        elif idx < len(self.children)-1 and len(self.children[idx+1].keys) >= t:
            # Borrow from right
            right = self.children[idx+1]
            child = self.children[idx]
            child.keys.append(self.keys[idx])
            if not child.leaf:
                child.children.append(right.children.pop(0))
            self.keys[idx] = right.keys.pop(0)
        else:
            # Merge with a sibling
            if idx < len(self.children)-1:
                self._merge(idx)
            else:
                self._merge(idx-1)

    # DELETE from subtree rooted here. Assumes this node is non-empty.
    def delete_nonempty(self, k: Any) -> None:
        t = self.t
        idx = 0
        while idx < len(self.keys) and k > self.keys[idx]:
            idx += 1

        # Case 1: key present in this node
        if idx < len(self.keys) and self.keys[idx] == k:
            if self.leaf:
                # 1a: key in leaf
                del self.keys[idx]
                return
            # 1b: key in internal node
            if len(self.children[idx].keys) >= t:
                pred = self._pred(idx)
                self.keys[idx] = pred
                self.children[idx].delete_nonempty(pred)
            elif len(self.children[idx+1].keys) >= t:
                succ = self._succ(idx)
                self.keys[idx] = succ
                self.children[idx+1].delete_nonempty(succ)
            else:
                self._merge(idx)
                self.children[idx].delete_nonempty(k)
            return

        # Case 2: key not present here
        if self.leaf:
            return  # key not found
        # Ensure the child we descend to has at least t keys
        if len(self.children[idx].keys) == t - 1:
            self._fill(idx)
            # After fill, structure may have changed. Decide next child:
            if idx > len(self.keys):   # merged with left
                idx -= 1
        self.children[idx].delete_nonempty(k)

# ------------------------- Tree -------------------------
class BTree:
    def __init__(self, t: int):
        if t < 2:
            raise ValueError("Minimum degree t must be ≥ 2")
        self.t = t
        self.root = BTreeNode(t=t, leaf=True)

    def search(self, k: Any) -> Optional[Tuple[BTreeNode, int]]:
        return self.root.search(k)

    # INSERT (B-TREE-INSERT)
    def insert(self, k: Any) -> None:
        r = self.root
        if len(r.keys) == 2*self.t - 1:
            s = BTreeNode(t=self.t, leaf=False, keys=[], children=[r])
            s.split_child(0)
            self.root = s
            s.insert_nonfull(k)
        else:
            r.insert_nonfull(k)

    # DELETE (top-level)
    def delete(self, k: Any) -> None:
        if not self.root.keys and self.root.leaf:
            return
        self.root.delete_nonempty(k)
        # Shrink height if root became empty and is internal
        if not self.root.keys and not self.root.leaf:
            self.root = self.root.children[0]

    # Utilities
    def inorder(self) -> List[Any]:
        out: List[Any] = []
        def dfs(x: BTreeNode):
            if x.leaf:
                out.extend(x.keys)
            else:
                for i, key in enumerate(x.keys):
                    dfs(x.children[i])
                    out.append(key)
                dfs(x.children[-1])
        dfs(self.root)
        return out

    def __contains__(self, k: Any) -> bool:
        return self.search(k) is not None


![b-tree](./img/b-trees.png)

In [28]:
## Now build and display a b tree whose keys are the consonants of English (as in CLRS Fig 18.1)

bt = BTree(t=2)
for k in ['F', 'S', 'Q', 'K', 'C']:
    bt.insert(k)

diagram(bt, method="browser")

'/var/folders/80/rtptthbx3zq0tb06wwzmck_40000gq/T/tmpst883gpn.html'

## Performance

In [29]:
STRUCTURE = "btree"
import string
for size in SIZES:
    values = random.sample(string.ascii_uppercase + string.ascii_lowercase, size)

    b = BTree(2)
    for val in values:
        b.insert(val)

    diagram(b, method="browser", perf_path=get_perf_path(STRUCTURE, size), perf_iterations=PI, headless=True, timeout=400)

Running 20 iterations (timeout: 400s)...
  Using custom timeout of 400s
  Progress: 20/20 iterations (10.0s elapsed)
✓ Headless benchmark completed: 20 iterations
  Generate Layout: 22.76ms avg
  Render Layout: 119.29ms avg
  Total Time: 142.66ms avg
  Metrics saved to: spytial_perf_btree_5.json
Running 20 iterations (timeout: 400s)...
  Using custom timeout of 400s
  Progress: 20/20 iterations (10.0s elapsed)
✓ Headless benchmark completed: 20 iterations
  Generate Layout: 37.01ms avg
  Render Layout: 167.06ms avg
  Total Time: 204.87ms avg
  Metrics saved to: spytial_perf_btree_10.json
Running 20 iterations (timeout: 400s)...
  Using custom timeout of 400s
  Progress: 20/20 iterations (22.8s elapsed)
✓ Headless benchmark completed: 20 iterations
  Generate Layout: 181.11ms avg
  Render Layout: 854.70ms avg
  Total Time: 1036.98ms avg
  Metrics saved to: spytial_perf_btree_25.json
Running 20 iterations (timeout: 400s)...
  Using custom timeout of 400s
  Progress: 20/20 iterations (95.

# van Emde Boas (vEB) Tree (CLRS Ch. 20)


In [30]:
## VEB Trees Code Here ###
from __future__ import annotations
from dataclasses import dataclass, field
from typing import Optional, Dict

def is_power_of_two(n: int) -> bool:
    return n > 0 and (n & (n - 1)) == 0

def upper_sqrt(u: int) -> int:
    # 2^{ceil(lg u / 2)}
    k = u.bit_length() - 1
    return 1 << ((k + 1) // 2)

def lower_sqrt(u: int) -> int:
    # 2^{floor(lg u / 2)}
    k = u.bit_length() - 1
    return 1 << (k // 2)

@dataclass
@hideAtom(selector="NoneType + int + {d : dict | no (d.kv)}")
@attribute(field="min")
@attribute(field="max")
@attribute(field="u")
@orientation(selector="(summary & (VEB->VEB)) + (cluster & (VEB -> { d: dict | some d.kv })) + kv", directions=["below"])
class VEB:
    u: int
    min: Optional[int] = None
    max: Optional[int] = None
    summary: Optional["VEB"] = None
    cluster: Dict[int, "VEB"] = field(default_factory=dict)

    def __post_init__(self):
        if not is_power_of_two(self.u):
            raise ValueError("u must be a power of two")
        if self.u < 2:
            raise ValueError("u must be >= 2")

    # index math
    def _high(self, x: int) -> int:
        return x // lower_sqrt(self.u)

    def _low(self, x: int) -> int:
        return x % lower_sqrt(self.u)

    def _index(self, high: int, low: int) -> int:
        return high * lower_sqrt(self.u) + low

    # base checks
    def _empty(self) -> bool:
        return self.min is None

    # queries
    def member(self, x: int) -> bool:
        if self.min == x or self.max == x:
            return True
        if self.u == 2:
            return False
        c = self._high(x)
        if c in self.cluster:
            return self.cluster[c].member(self._low(x))
        return False

    def minimum(self) -> Optional[int]:
        return self.min

    def maximum(self) -> Optional[int]:
        return self.max

    # insert into empty VEB
    def _empty_insert(self, x: int) -> None:
        self.min = x
        self.max = x

    # insert (CLRS VEB-INSERT)
    def insert(self, x: int) -> None:
        if self._empty():
            self._empty_insert(x)
            return
        if x < self.min:  # swap into min
            x, self.min = self.min, x
        if self.u > 2:
            c = self._high(x)
            i = self._low(x)
            if c not in self.cluster or self.cluster[c]._empty():
                # create cluster if absent
                if c not in self.cluster:
                    self.cluster[c] = VEB(lower_sqrt(self.u))
                if self.summary is None:
                    self.summary = VEB(upper_sqrt(self.u))
                # first insertion into this cluster
                self.summary.insert(c)
                self.cluster[c]._empty_insert(i)
            else:
                self.cluster[c].insert(i)
        if x > self.max:
            self.max = x

    # delete (CLRS VEB-DELETE)
    def delete(self, x: int) -> None:
        if self.min == self.max:
            # single element
            if self.min == x:
                self.min = None
                self.max = None
            return

        if self.u == 2:
            # u == 2 and two distinct values exist
            if x == 0:
                self.min = 1
            else:
                self.min = 0
            self.max = self.min
            return

        if x == self.min:
            # replace min with first element of the first non-empty cluster
            assert self.summary is not None
            first_cluster = self.summary.minimum()
            if first_cluster is None:
                # should not happen because >1 elements exist
                return
            x = self._index(first_cluster, self.cluster[first_cluster].minimum())
            self.min = x

        # delete x from its cluster
        c = self._high(x)
        i = self._low(x)
        if c in self.cluster:
            self.cluster[c].delete(i)
            if self.cluster[c]._empty():
                # remove cluster from summary
                self.summary.delete(c)
                # optional: free empty cluster to save space
                # del self.cluster[c]

                # if all clusters empty, max is min
                if self.summary._empty():
                    self.summary = None
                    self.max = self.min
                else:
                    # recompute max from last non-empty cluster
                    last_cluster = self.summary.maximum()
                    self.max = self._index(last_cluster, self.cluster[last_cluster].maximum())
            else:
                # cluster still has elements; update max if needed
                if x == self.max:
                    last_cluster = self._high(self.max)
                    self.max = self._index(last_cluster, self.cluster[last_cluster].maximum())

    # successor (CLRS VEB-SUCCESSOR)
    def successor(self, x: int) -> Optional[int]:
        if self._empty():
            return None
        if self.u == 2:
            if x == 0 and self.max == 1:
                return 1
            return None

        if x < self.min:
            return self.min

        c = self._high(x)
        i = self._low(x)

        # try in same cluster
        if c in self.cluster and not self.cluster[c]._empty() and i < self.cluster[c].maximum():
            offset = self.cluster[c].successor(i)
            return self._index(c, offset)

        # find next cluster
        if self.summary is None:
            return None
        next_cluster = self.summary.successor(c)
        if next_cluster is None:
            return None
        return self._index(next_cluster, self.cluster[next_cluster].minimum())

    # predecessor (CLRS VEB-PREDECESSOR)
    def predecessor(self, x: int) -> Optional[int]:
        if self._empty():
            return None
        if self.u == 2:
            if x == 1 and self.min == 0:
                return 0
            return None

        if x > self.max:
            return self.max

        c = self._high(x)
        i = self._low(x)

        # try in same cluster
        if c in self.cluster and not self.cluster[c]._empty() and i > self.cluster[c].minimum():
            offset = self.cluster[c].predecessor(i)
            return self._index(c, offset)

        # find previous cluster
        if self.summary is None:
            if x > self.min:
                return self.min
            return None
        prev_cluster = self.summary.predecessor(c)
        if prev_cluster is None:
            if x > self.min:
                return self.min
            return None
        return self._index(prev_cluster, self.cluster[prev_cluster].maximum())


![veb](./img/veb-tree.png)

In [31]:
# Build a VEB tree for the set {2,3,4,5,7,14,15}
S = [2, 3, 4, 5, 7, 14, 15]
u = 16  # universe size must be a power of two >= max(S)+1

veb = VEB(u)
for x in S:
    veb.insert(x)

print("min:", veb.minimum(), "max:", veb.maximum())
print("members:", sorted(x for x in S if veb.member(x)))
diagram(veb)


min: 2 max: 15
members: [2, 3, 4, 5, 7, 14, 15]


## Performance

In [32]:
STRUCTURE = "vebtree"
for size in SIZES:
    # vEB requires universe size u as power of two; use 64 for sufficient range
    u = 64
    values = random.sample(range(0, u), size)

    veb = VEB(u=u)
    for val in values:
        veb.insert(val)

    diagram(veb, method="browser", perf_path=get_perf_path(STRUCTURE, size), perf_iterations=PI, headless=True, timeout=400)

Running 20 iterations (timeout: 400s)...
  Using custom timeout of 400s
  Progress: 20/20 iterations (10.0s elapsed)
✓ Headless benchmark completed: 20 iterations
  Generate Layout: 11.97ms avg
  Render Layout: 63.09ms avg
  Total Time: 75.89ms avg
  Metrics saved to: spytial_perf_vebtree_5.json
Running 20 iterations (timeout: 400s)...
  Using custom timeout of 400s
  Progress: 20/20 iterations (10.0s elapsed)
✓ Headless benchmark completed: 20 iterations
  Generate Layout: 14.63ms avg
  Render Layout: 108.50ms avg
  Total Time: 124.21ms avg
  Metrics saved to: spytial_perf_vebtree_10.json
Running 20 iterations (timeout: 400s)...
  Using custom timeout of 400s
  Progress: 20/20 iterations (10.0s elapsed)
✓ Headless benchmark completed: 20 iterations
  Generate Layout: 34.34ms avg
  Render Layout: 266.65ms avg
  Total Time: 302.85ms avg
  Metrics saved to: spytial_perf_vebtree_25.json
Running 20 iterations (timeout: 400s)...
  Using custom timeout of 400s
  Progress: 20/20 iterations (2

# Interval Trees


In [33]:

# Interval node augments RBNode. We use key == low for ordering.
@attribute(field="low")
@attribute(field="high")
@attribute(field="max")
class IntervalNode(RBNode):
    def __init__(self, low, high, color=RED, left=None, right=None, parent=None):
        super().__init__(key=low, color=color, left=left if left is not None else NIL,
                         right=right if right is not None else NIL,
                         parent=parent if parent is not None else NIL)
        self.low = low
        self.high = high
        self.max = high

class IntervalTree(RBTree):
    def __init__(self):
        super().__init__()
        # Ensure NIL has a max field used by augmentation
        if not hasattr(NIL, "max"):
            setattr(NIL, "max", 0)

    # ---- augmentation helpers ----
    def _update(self, x: IntervalNode):
        if x is NIL:
            return
        lm = x.left.max if x.left is not NIL else 0
        rm = x.right.max if x.right is not NIL else 0
        x.max = max(x.high, lm, rm)

    def _update_up(self, x: IntervalNode):
        while x is not NIL:
            old = x.max
            self._update(x)
            if x.max == old:
                break
            x = x.parent

    # ---- rotations with max maintenance ----
    def left_rotate(self, x: IntervalNode):
        y = x.right
        assert y is not NIL
        # standard rotate
        x.right = y.left
        if y.left is not NIL:
            y.left.parent = x
        y.parent = x.parent
        if x.parent is NIL:
            self.root = y
        elif x is x.parent.left:
            x.parent.left = y
        else:
            x.parent.right = y
        y.left = x
        x.parent = y
        # update augmented fields bottom-up
        self._update(x)
        self._update(y)
        self._update_up(y.parent)

    def right_rotate(self, y: IntervalNode):
        x = y.left
        assert x is not NIL
        # standard rotate
        y.left = x.right
        if x.right is not NIL:
            x.right.parent = y
        x.parent = y.parent
        if y.parent is NIL:
            self.root = x
        elif y is y.parent.left:
            y.parent.left = x
        else:
            y.parent.right = x
        x.right = y
        y.parent = x
        # update augmented fields bottom-up
        self._update(y)
        self._update(x)
        self._update_up(x.parent)

    # ---- interval overlap ----
    @staticmethod
    def _overlap(a_low, a_high, b_low, b_high) -> bool:
        return not (a_high < b_low or b_high < a_low)

    # ---- search one overlapping interval ----
    def interval_search(self, low: int, high: int):
        x = self.root
        while x is not NIL and not self._overlap(low, high, x.low, x.high):
            if x.left is not NIL and x.left.max >= low:
                x = x.left
            else:
                x = x.right
        return None if x is NIL else x

    # ---- insert (creates IntervalNode, maintains max) ----
    def insert(self, low: int, high: int):
        z = IntervalNode(low, high, color=RED)
        y = NIL
        x = self.root
        # BST insert by low key
        while x is not NIL:
            y = x
            x = x.left if z.key < x.key else x.right
        z.parent = y
        if y is NIL:
            self.root = z
        elif z.key < y.key:
            y.left = z
        else:
            y.right = z
        # fix max on path up
        self._update_up(z)
        # RB fix
        self._insert_fixup(z)
        return z

    # ---- delete (CLRS RB-DELETE + max maintenance). Expects a node handle. ----
    def delete(self, z: IntervalNode):
        # local helpers
        def transplant(u, v):
            if u.parent is NIL:
                self.root = v
            elif u is u.parent.left:
                u.parent.left = v
            else:
                u.parent.right = v
            v.parent = u.parent

        def minimum(x):
            while x.left is not NIL:
                x = x.left
            return x

        y = z
        y_orig_color = y.color
        if z.left is NIL:
            x = z.right
            transplant(z, z.right)
            start = x.parent
        elif z.right is NIL:
            x = z.left
            transplant(z, z.left)
            start = x.parent
        else:
            y = minimum(z.right)
            y_orig_color = y.color
            x = y.right
            if y.parent is z:
                x.parent = y
            else:
                transplant(y, y.right)
                y.right = z.right
                y.right.parent = y
            transplant(z, y)
            y.left = z.left
            y.left.parent = y
            y.color = z.color
            # update y first (its children changed)
            self._update(y)
            start = y

        # update augmented info upward from the structural change
        self._update_up(start)

        if y_orig_color == BLACK:
            self._delete_fixup(x)

    def _delete_fixup(self, x: IntervalNode):
        while x is not self.root and x.color is BLACK:
            if x is x.parent.left:
                w = x.parent.right
                if w.color is RED:
                    w.color = BLACK
                    x.parent.color = RED
                    self.left_rotate(x.parent)
                    w = x.parent.right
                if w.left.color is BLACK and w.right.color is BLACK:
                    w.color = RED
                    x = x.parent
                else:
                    if w.right.color is BLACK:
                        w.left.color = BLACK
                        w.color = RED
                        self.right_rotate(w)
                        w = x.parent.right
                    w.color = x.parent.color
                    x.parent.color = BLACK
                    w.right.color = BLACK
                    self.left_rotate(x.parent)
                    x = self.root
            else:
                w = x.parent.left
                if w.color is RED:
                    w.color = BLACK
                    x.parent.color = RED
                    self.right_rotate(x.parent)
                    w = x.parent.left
                if w.right.color is BLACK and w.left.color is BLACK:
                    w.color = RED
                    x = x.parent
                else:
                    if w.left.color is BLACK:
                        w.right.color = BLACK
                        w.color = RED
                        self.left_rotate(w)
                        w = x.parent.left
                    w.color = x.parent.color
                    x.parent.color = BLACK
                    w.left.color = BLACK
                    self.right_rotate(x.parent)
                    x = self.root
        x.color = BLACK
        # restore augmentation on the path up
        self._update_up(x)

    # ---- optional: exact lookup by (low, high) ----
    def find_exact(self, low: int, high: int):
        x = self.root
        while x is not NIL:
            if low == x.low and high == x.high:
                return x
            x = x.left if low < x.low else x.right
        return None


![interval-trees](./img/interval-tree.png)

In [34]:
it = IntervalTree()
for low, high in [(16,21), (8,9), (25,30), (5,8), (15,23), (17,19), (26,26), (0,3), (6,10), (19,20)]:
    it.insert(low, high)
diagram(it)

## Performance Testing

In [35]:
STRUCTURE = "intervaltree"
for size in SIZES:
    # Generate random intervals
    intervals = []
    for _ in range(size):
        low = random.randint(0, 1000)
        high = random.randint(low + 1, 1000 + 10)  # ensure high > low
        intervals.append((low, high))

    it = IntervalTree()
    for low, high in intervals:
        it.insert(low, high)

    diagram(it, method="browser", perf_path=get_perf_path(STRUCTURE, size), perf_iterations=PI, headless=True, timeout=400)

Running 20 iterations (timeout: 400s)...
  Using custom timeout of 400s
  Progress: 20/20 iterations (10.0s elapsed)
✓ Headless benchmark completed: 20 iterations
  Generate Layout: 15.97ms avg
  Render Layout: 26.45ms avg
  Total Time: 43.38ms avg
  Metrics saved to: spytial_perf_intervaltree_5.json
Running 20 iterations (timeout: 400s)...
  Using custom timeout of 400s
  Progress: 20/20 iterations (10.0s elapsed)
✓ Headless benchmark completed: 20 iterations
  Generate Layout: 16.43ms avg
  Render Layout: 43.52ms avg
  Total Time: 61.25ms avg
  Metrics saved to: spytial_perf_intervaltree_10.json
Running 20 iterations (timeout: 400s)...
  Using custom timeout of 400s
  Progress: 20/20 iterations (10.0s elapsed)
✓ Headless benchmark completed: 20 iterations
  Generate Layout: 18.44ms avg
  Render Layout: 97.73ms avg
  Total Time: 118.37ms avg
  Metrics saved to: spytial_perf_intervaltree_25.json
Running 20 iterations (timeout: 400s)...
  Using custom timeout of 400s
  Progress: 20/20 i